<p style = 'font-size:18px;color:black'> This POC demonstrates how to integrate Twilio and OpenAI to make automated phone calls that deliver a message generated by OpenAI's language model. The script generates a custom message using OpenAI's GPT-3.5-turbo model and then makes a phone call via Twilio to deliver the message to a specified recipient </p>

<!DOCTYPE html>
<html>

<head>
    <style>
        table {
            font-family: Arial, sans-serif;
            border-collapse: collapse;
            width: 100%;
        }

        th {
            background-color: #f2f2f2;
            text-align: left;
            padding: 8px;
        }

        td {
            border-bottom: 1px solid #ddd;
            padding: 8px;
        }
    </style>
</head>

<body>

    <h2>Script Workflow</h2>

    <table>
        <tr>
            <th>Step</th>
            <th>Description</th>
        </tr>
        <tr>
            <td>Initialize Clients</td>
            <td>Set up the Twilio and OpenAI clients with the necessary credentials.</td>
        </tr>
        <tr>
            <td>Generate Text</td>
            <td>Use OpenAI to generate a custom message based on a predefined prompt.</td>
        </tr>
        <tr>
            <td>Handle Rate Limits</td>
            <td>Implement retry logic to manage API rate limits.</td>
        </tr>
        <tr>
            <td>Make the Call</td>
            <td>Use Twilio to place a call to the recipient's phone number, delivering the generated message.</td>
        </tr>
        <tr>
            <td>Output Call SID</td>
            <td>Print the unique identifier (SID) for the created call for tracking and verification.</td>
        </tr>
    </table>

</body>

</html>


<p style='font-size:20px; color:purple;'><b><u>Script Workflow</u></b></p>
<ul>
    <li><b>Initialize Clients:</b> Set up the Twilio and OpenAI clients with the necessary credentials.</li>
    <li><b>Generate Text:</b> Use OpenAI to generate a custom message based on a predefined prompt.</li>
    <li><b>Handle Rate Limits:</b> Implement retry logic to manage API rate limits.</li>
    <li><b>Make the Call:</b> Use Twilio to place a call to the recipients phone number, delivering the generated message.</li>
    <li><b>Output Call SID:</b> Print the unique identifier (SID) for the created call for tracking and verification.</li>
</ul>


In [4]:
pip install openai==0.28

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/76.5 kB ? eta -:--:--
   -------------------------------- ------- 61.4/76.5 kB 656.4 kB/s eta 0:00:01
   ---------------------------------------- 76.5/76.5 kB 705.4 kB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.30.5
    Uninstalling openai-1.30.5:
      Successfully uninstalled openai-1.30.5
Note: you may need to restart the kernel to use updated packages.


In [23]:
from twilio.rest import Client  #make calls using Twilio's API
import openai  #interact with OpenAI's AP
import time

# Twilio credentials
twilio_account_sid = 'YOUR_TWILIO_ACCOUNT_SID'  #authenticate our requests to Twilio's API.
twilio_auth_token = 'YOUR_TWILIO_AUTH_TOKEN'    #serves as a password to authenticate API requests
twilio_phone_number = '+14128992270'            #from which we want to make calls 

# OpenAI API key
openai_api_key = 'OPENAI_API_KEY' #authenticates your requests to OpenAI's API

# Initialize Twilio client
client = Client(twilio_account_sid, twilio_auth_token) #used to interact with Twilio's API

# Initialize OpenAI
openai.api_key = openai_api_key

# Function to generate text with OpenAI
def generate_text(prompt, retries=3):    #prompt-->input we provide,retrices-->number of times function will attempt to generate text
    for i in range(retries):
        try:
            response = openai.ChatCompletion.create(   #sends a request to the OpenAI API to generate text
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},  #message is from the system
                    {"role": "user", "content": prompt}
                ]
            )
            return response.choices[0].message['content'].strip() #extracts generated text from response & returns after stripping any whitespace
        except openai.error.RateLimitError:
            if i < retries - 1:
                print("Rate limit exceeded, retrying...")
                time.sleep(5)  # Wait for 5 seconds before retrying
            else:
                print("Rate limit exceeded, no more retries.")
                raise
'''error.Ratelimiterror-->this error occurs when the rate at which requests are made 
   exceeds the rate limit allowed by OpenAI for the specific API endpoint being accessed.'''

# Generate text to be spoken in the call
try:
    text_to_say = generate_text("Hello! This is a test call using Twilio and OpenAI. How can I assist you today?")
except openai.error.RateLimitError:   
    text_to_say = "I'm sorry, I'm currently experiencing high demand. Please try again later."

# Make the call using Twilio
call = client.calls.create(  #initiates a call
    twiml=f'<Response><Say>{text_to_say}</Say></Response>', #creates a TWiML response 
    to='to_number',
    from_=twilio_phone_number
)

print(f"Call SID: {call.sid}")


Rate limit exceeded, retrying...
Rate limit exceeded, retrying...
Rate limit exceeded, no more retries.
Call SID: CA90e4bca2f2cf6c35df01b7e0d241e71a
